## Dataset létrehozása fasta file-okból indulva.
Input: folder, amiben a fasta file-ok vannak.
Output: X mátrix, amiben a tokenizált vektorok vannak. 
Opcinálisan egy adatbázis a szegmensekről, gyakorlatban a szekvencia adatot nem fogjuk tárolni, csak a koordinátákat


In [2]:
%load_ext autoreload
%autoreload 2
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *

    
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 30)
#pd.set_option('display.width', 4000)
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', True)
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2
defconfig = SeqConfig()
         
segmentation_params = defconfig.get_set_segmentation_parameters()
tokenization_params = defconfig.get_and_set_tokenization_params({'shift':2})             
comp_params = defconfig.get_set_computational_paramters()

num_cores = comp_params['cpu_cores_for_tokenization']
batch_size = comp_params['batch_size_tokenization']
numpy_dtype=comp_params['np_tokentype'] # Note that if you use kmer>7 then other is recommended
           
input_fasta_dir = join(current_path, 'data/sample_data/pretraining')

input_fasta_files = [join(input_fasta_dir, file) for file in get_non_empty_files(input_fasta_dir)]
contigs = load_contigs(input_fasta_files,IsAddHeader=True,AsDataFrame=True, adding_reverse_complement=False)
contigs['sequence_id'] = list(range(len(contigs)))
contigs = contigs[['sequence', 'sequence_id']]
segment_db = segment_sequences(contigs, segmentation_params, AsDataFrame=True)

tokenized = batch_tokenize_segments_with_ids(segment_db, tokenization_params, num_cores, batch_size, numpy_dtype)
    

2023-08-11 08:00:49,942 - INFO - Loading sequence data into memory!
2023-08-11 08:00:49,958 - INFO - Checking input DataFrame!
2023-08-11 08:00:49,958 - INFO - Checking input sequence_id is valid primary key in the DataFrame


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
/home/ligeti/github/prokbert/src/prokbert


2023-08-11 08:00:50,141 - INFO - Tokenization of a list of segments


In [3]:
shift = tokenization_params['shift']
expected_max_token = max(len(arr) for arrays in tokenized.values() for arr in arrays)
X, torchdb = get_rectangular_array_from_tokenized_dataset(tokenized,
                                                          tokenization_params['shift'],
                                                          expected_max_token, numpy_dtype = numpy_dtype)


2023-08-11 08:00:52,488 - INFO - Doing randomization!
2023-08-11 08:00:52,491 - INFO - Tuncating all zeros column


In [17]:
### act_segment = 'ATCGCTATGGT'
tokenized_seg, kmers = lca_tokenize_segment(act_segment, tokenization_params)
kmers
tokenized_seg

[['ATCGCT', 'CGCTAT', 'CTATGG'], ['TCGCTA', 'GCTATG', 'TATGGT']]

[[2, 876, 1656, 1855, 3], [2, 3489, 2515, 3312, 3]]

In [16]:
lines = pretty_print_overlapping_sequence(act_segment, kmers[0], tokenization_params)
print(lines)

2023-08-11 08:10:04,048 - INFO - Nr. line to cover the seq:  4


    ATCGCTATGGT
0.  ATCGCT
1.    CGCTAT
2.      CTATGG
3.        
